In [181]:
import numpy as np
import pandas as pd
from datetime import datetime
import tqdm

checkin_file = "../dataset/loc-gowalla_totalCheckins.txt"
df = pd.read_csv(checkin_file, sep='\t', header=None)
df.columns = ["user", "time", "latitude", "longitude", "poi"]
df = df[['user', 'time', 'poi']]

In [182]:
len(df)

6442892

In [183]:
prev_cnt = 0
curr_cnt = len(df)
while prev_cnt != curr_cnt:
    prev_cnt = curr_cnt
    df = df[df.groupby('user').user.transform(len) > 5]
    df = df[df.groupby('poi').poi.transform(len) > 5]
    curr_cnt = len(df)

In [184]:
len(df)

4293047

In [186]:
poi2id = np.load("./npy/poi2id.npy").item()
df['poi'] = df['poi'].apply(lambda x: poi2id[x] if poi2id.get(x) != None else 0)
df = df[df['poi'] != 0]
df['time'] = df['time'].apply(lambda x: (datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ")-datetime(2009,1,1)).total_seconds()\
                              /360)  # hour

In [187]:
np.max(df.poi)

15233

In [188]:
user2id = {'unk':0}
id2user = [0]
for target_idx in tqdm.tqdm(xrange(len(df))):
    (user, time, poi) = df.iloc[target_idx]
    if user2id.get(user) == None:
        user2id[user] = len(id2user)
        id2user.append(user)
    if len(id2user) == 4627:
        break
df['user'] = df['user'].apply(lambda x: user2id[x] if user2id.get(x) != None else 0)
df = df[df['user'] != 0]

 43%|████▎     | 242208/561673 [00:31<00:41, 7721.47it/s]

In [189]:
len(df.groupby('user')),len(df)

(4626, 242873)

In [190]:
train_user = []
train_context = []
train_target = []
valid_user = []
valid_context = []
valid_target = []
test_user = []
test_context = []
test_target = []

tow = 6
prev_user = df['user'][0]
user_user = []
user_context = []
user_target = []
for target_idx in tqdm.tqdm(xrange(len(df))):
    (user, time, poi) = df.iloc[target_idx]
    if prev_user != user:
        prev_user = user
        if len(user_user) > 20:
            train_thr = int(len(user_user)*0.9)
            valid_thr = int(len(user_user)*0.95)
            train_user += user_user[:train_thr]
            train_context += user_context[:train_thr]
            train_target += user_target[:train_thr]
            valid_user += user_user[train_thr:valid_thr]
            valid_context += user_context[train_thr:valid_thr]
            valid_target += user_target[train_thr:valid_thr]
            test_user += user_user[valid_thr:]
            test_context += user_context[valid_thr:]
            test_target += user_target[valid_thr:]
        elif len(user_user) > 0:
            train_user += user_user
            train_context += user_context
            train_target += user_target    
        user_user = []
        user_context = []
        user_target = []
        #print train_user, train_context, train_target
    
    context = []
    for context_idx in xrange(target_idx+1, len(df)):
        (c_user, c_time, c_poi) = df.iloc[context_idx]
        if user == c_user and (time-tow) < c_time:
                context.append(c_poi)
        else:
            break
    if context:
        user_user.append(user)
        user_context.append(context)
        user_target.append(poi)
        
if len(user_user) > 20:
    train_thr = int(len(user_user)*0.9)
    valid_thr = int(len(user_user)*0.95)
    train_user += user_user[:train_thr]
    train_context += user_context[:train_thr]
    train_target += user_target[:train_thr]
    valid_user += user_user[train_thr:valid_thr]
    valid_context += user_context[train_thr:valid_thr]
    valid_target += user_target[train_thr:valid_thr]
    test_user += user_user[valid_thr:]
    test_context += user_context[valid_thr:]
    test_target += user_target[valid_thr:]
elif len(user_user) > 0:
    train_user += user_user
    train_context += user_context
    train_target += user_target


  7%|▋         | 16728/242873 [00:08<01:51, 2022.11it/s]

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/yongqyu/yongqyu/local/lib/python2.7/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/yongqyu/yongqyu/lib/python2.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 15%|█▌        | 36785/242873 [00:16<01:33, 2212.15it/s]


 32%|███▏      | 78502/242873 [00:34<01:11, 2300.71it/s]


 48%|████▊     | 117764/242873 [00:51<00:54, 2288.50it/s]


 65%|██████▌   | 158646/242873 [01:07<00:35, 2363.15it/s]


 83%|████████▎ | 202533/242873 [01:23<00:16, 2433.53it/s]


100%|██████████| 242873/242873 [01:36<00:00, 2518.41it/s]

In [191]:
len(train_user), len(train_context), len(train_target), len(valid_user), len(valid_context), len(valid_target), len(test_user), len(test_context), len(test_target)

(53359, 53359, 53359, 2336, 2336, 2336, 2677, 2677, 2677)

In [192]:
len_context = []
for i, context in enumerate(train_context):
    len_context.append(len(context))
print np.max(len_context), np.mean(len_context), np.median(len_context), np.min(len_context)
for i, context in enumerate(valid_context):
    len_context.append(len(context))
for i, context in enumerate(test_context):
    len_context.append(len(context))
len_context.sort()
print np.max(len_context), np.mean(len_context), np.median(len_context), np.min(len_context)

64 4.00256751438 2.0 1
64 3.94891386281 2.0 1


In [193]:
print len(len_context)
print len_context[int(len(len_context)*0.99)]

58372
26


In [194]:
maxlen_context = 32
for i, context in enumerate(train_context):
    if len(context) < maxlen_context:
        train_context[i] += ([0]*(maxlen_context-len(context)))
    elif len(context) > maxlen_context:
        train_context[i] = context[:maxlen_context]
for i, context in enumerate(valid_context):
    if len(context) < maxlen_context:
        valid_context[i] += ([0]*(maxlen_context-len(context)))
    elif len(context) > maxlen_context:
        valid_context[i] = context[:maxlen_context]
for i, context in enumerate(test_context):
    if len(context) < maxlen_context:
        test_context[i] += ([0]*(maxlen_context-len(context)))
    elif len(context) > maxlen_context:
        test_context[i] = context[:maxlen_context]

In [195]:
len_context = []
for context in test_context:
    len_context.append(len(context))
print np.max(len_context), np.mean(len_context), np.median(len_context), np.min(len_context)

32 32.0 32.0 32


In [196]:
np.save('./npy/train_context.npy', train_context)
np.save('./npy/valid_context.npy', valid_context)
np.save('./npy/test_context.npy', test_context)
np.save('./npy/user2id.npy', user2id)
np.save('./npy/id2user.npy', id2user)

In [197]:
np.save('./npy/train_user.npy', train_user)
np.save('./npy/valid_user.npy', valid_user)
np.save('./npy/test_user.npy', test_user)
np.save('./npy/train_target.npy', train_target)
np.save('./npy/valid_target.npy', valid_target)
np.save('./npy/test_target.npy', test_target)

In [101]:
len(id2user)

4627